In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from tqdm import tqdm
from time import sleep
from os import listdir
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation as FA
import math
from decimal import Decimal, ROUND_HALF_UP

import Sources.Utils as UtilsTool
import Sources.Attacks as AttacksTool
import Sources.Data as DataTool

from Sources.Models import RNNModel, GRUModel, LSTMModel

import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

device = 'cuda'
DisplayGraph = 0
TrainMode = 0
OptimizeParameters = 1

DatasetPath = 'DatasetRUL/'
ModelPath = 'Models/'
NormalOutputPath = 'ResultsRUL/Normal/'
AdversarialOutputPath = 'ResultsRUL/Adversarial/'
OptimalParametersPath = 'ResultsRUL/OptimalParameters/'

#does not consider a large window TODO
#Save MAillard L2, L0, Noramlized datas TODO
#Clean repository TODO
window = 50

In [3]:
listdir(DatasetPath)

df_train = pd.read_csv(DatasetPath + 'train_FD001.txt', header = None, sep = ' ')
df_test = pd.read_csv(DatasetPath + 'test_FD001.txt', header = None, sep = ' ')
rul_test = pd.read_csv(DatasetPath + 'RUL_FD001.txt', header = None)

In [4]:
df_train, df_test, rul_test = DataTool.SetDatasets(df_train, df_test, rul_test)

In [5]:
minmax_dict = DataTool.GetMinMaxDictionnary(df_train)
Ndf_train  = DataTool.NormalizeDataset(df_train, minmax_dict)    
Ndf_test  = DataTool.NormalizeDataset(df_test, minmax_dict)   

In [6]:
n_features = len([c for c in Ndf_train.columns if 's' in c])


In [7]:
np.random.seed(5)
torch.manual_seed(5)

units = np.arange(1,101)
train_units = list(np.random.choice(units, 80, replace = False))
val_units = list(set(units) - set(train_units))

train_data = Ndf_train[Ndf_train['unit'].isin(train_units)].copy()
val_data = Ndf_train[Ndf_train['unit'].isin(val_units)].copy()

train_indices = list(train_data[(train_data['rul'] >= (window - 1)) & (train_data['time'] > 10)].index)
val_indices = list(val_data[(val_data['rul'] >= (window - 1)) & (val_data['time'] > 10)].index)

train = DataTool.data(train_indices, Ndf_train)
val = DataTool.data(val_indices, Ndf_train)

trainloader = DataLoader(train, batch_size = 64, shuffle = True)
valloader = DataLoader(val, batch_size = len(val_indices), shuffle = True)

units = np.arange(1,101)

testdataset = DataTool.test(units, Ndf_test)
testloader = DataLoader(testdataset, batch_size = 100)

In [8]:
if TrainMode == 1:
    epochs = 30
    learning_rate= 0.003
    
    MyRNNModel = RNNModel(n_features, 12).to(device)
    UtilsTool.InitModel(MyRNNModel, device)
    UtilsTool.TrainModel(MyRNNModel, trainloader, valloader, epochs, learning_rate, device)        
    torch.save(MyRNNModel, ModelPath + 'MyRNNmodel.pth')

    MyGRUModel = GRUModel(n_features, 12).to(device)
    UtilsTool.InitModel(MyGRUModel, device)
    UtilsTool.TrainModel(MyGRUModel, trainloader, valloader, epochs, learning_rate, device)        
    torch.save(MyGRUModel, ModelPath + 'MyGRUmodel.pth')

    MyLSTMModel = LSTMModel(n_features, 12).to(device)
    UtilsTool.InitModel(MyLSTMModel, device)
    UtilsTool.TrainModel(MyLSTMModel, trainloader, valloader, epochs, learning_rate, device)     
    torch.save(MyLSTMModel, ModelPath + 'MyLSTMmodel.pth')
else:
    MyRNNModel = torch.load(ModelPath + 'MyRNNmodel.pth')
    MyGRUModel = torch.load(ModelPath + 'MyGRUmodel.pth')
    MyLSTMModel = torch.load(ModelPath + 'MyLSTMmodel.pth')

In [9]:
MyRNNModel = MyRNNModel.to(device)
MyGRUModel = MyGRUModel.to(device)
MyLSTMModel = MyLSTMModel.to(device)

![](./animation3.gif)

In [10]:
mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.test(MyRNNModel, testloader, device)
print(f'RNN: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_gru, y = UtilsTool.test(MyGRUModel, testloader, device)
print(f'GRU: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_lstm, y = UtilsTool.test(MyLSTMModel, testloader, device)
print(f'LSTM: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')

RNN: MSE:881.22, L1:23.44, RMSE:29.69
GRU: MSE:748.74, L1:19.82, RMSE:27.36
LSTM: MSE:609.01, L1:19.34, RMSE:24.68


In [11]:
if DisplayGraph == 1:
    UtilsTool.DisplayGraph(y_pred_rnn, y)
    UtilsTool.DisplayGraph(y_pred_gru, y)
    UtilsTool.DisplayGraph(y_pred_lstm, y)

In [12]:
testloaderAttack = DataLoader(testloader.dataset, batch_size=100, shuffle=False)
X, y = next(iter(testloaderAttack))
X, y = X.to(device), y.to(device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.1
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 100,
        "Epsilon": 0.002
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0001,
    }
}

print("RNN")
RNNAdversarialDataFgsm, RNNAdversarialDataBim, RNNAdversarialDataCW = AttacksTool.TestAttacks(MyRNNModel, X, y, AttacksParameters, device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.06
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 80,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.001,
    }
}

print("GRU")
GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.15
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 100,
        "Epsilon": 0.002
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0001,
    }
}

print("LSTM")
LSTMAdversarialDataFgsm, LSTMAdversarialDataBim, LSTMAdversarialDataCW = AttacksTool.TestAttacks(MyLSTMModel, X, y, AttacksParameters, device)
print()

RNN
FGSM: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 47.80316162109375, 'RMSE_adversarial': 0.09092852473258972, 'RMSE_pred': 42.452213287353516, 'L0': tensor(23.7200)}
BIM: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 28.27294158935547, 'RMSE_adversarial': 0.094657301902771, 'RMSE_pred': 60.857696533203125, 'L0': tensor(23.7000)}
CW: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 10.619772911071777, 'RMSE_adversarial': 0.10618610680103302, 'RMSE_pred': 77.25652313232422, 'L0': tensor(23.9800)}

GRU
FGSM: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 22.89347267150879, 'RMSE_adversarial': 0.0545102022588253, 'RMSE_pred': 61.21987533569336, 'L0': tensor(23.9400)}
BIM: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 8.817352294921875, 'RMSE_adversarial': 0.0562066026031971, 'RMSE_pred': 76.15291595458984, 'L0': tensor(23.8600)}
CW: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.2721460163593292, 'RMSE_adversarial': 0.055

In [13]:
UtilsTool.CreateDFSFiles(testloaderAttack, X, RNNAdversarialDataFgsm, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, RNNAdversarialDataBim, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, RNNAdversarialDataCW, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, GRUAdversarialDataFgsm, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, GRUAdversarialDataBim, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, GRUAdversarialDataCW, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, LSTMAdversarialDataFgsm, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, LSTMAdversarialDataBim, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, LSTMAdversarialDataCW, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/CW", device)

In [14]:
if DisplayGraph == 1:
    y_pred = MyRNNModel(RNNAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y)
    y_pred = MyRNNModel(RNNAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y)
    y_pred = MyRNNModel(RNNAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y)

In [15]:
if DisplayGraph == 1:
    y_pred = MyLSTMModel(LSTMAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y)
    y_pred = MyLSTMModel(LSTMAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y)
    y_pred = MyLSTMModel(LSTMAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y)

In [16]:
if DisplayGraph == 1:
    y_pred = MyGRUModel(GRUAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y)
    y_pred = MyGRUModel(GRUAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y)
    y_pred = MyGRUModel(GRUAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y)

In [17]:
class ChoiceLoss(nn.Module):
    def __init__(self, device):
        self.device = device
        super(ChoiceLoss, self).__init__()

    def forward(self, X, Z):
        Focus = torch.tensor([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
        Focus2D = Focus.unsqueeze(0).repeat(len(X[0]), 1)
        Focus2D = Focus2D.to(self.device)
        
        L2 = ((X - Z) ** 2)
        L2*= Focus2D
        
        loss = torch.sum(L2)
        
        return loss
 
class L0Loss(nn.Module):
    
    def tanh(self, x, c):
        return torch.tanh(x*c)
    
    def __init__(self, k, device):
        self.k=k
        self.device = device
        super(L0Loss, self).__init__()

    def forward(self, X, Z):
        Focus = torch.tensor([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
        Focus2D = Focus.unsqueeze(0).repeat(len(X[0]), 1)
        Focus2D = Focus2D.to(self.device)
        
        L2 = ((X - Z) ** 2)
        L2*= Focus2D
        Result = self.tanh(L2, self.k)
        Result2 = torch.sum(Result, dim=1)
        Result3 = self.tanh(Result2, self.k)

        loss = torch.sum(Result3)
        return loss

def MaillardL2(model, Objective, X, y, LearningRate, c, epochs, device):
    model = model.to(device)
    X = X.to(device)
    
    model.train()
    Adv = X.clone()
    Adv.requires_grad = True
    Optimizer = optim.Adam([Adv], lr=LearningRate)
    MyLoss = ChoiceLoss(device)
    MseLoss = nn.MSELoss()
    
    Adv = Adv.to(device)
    for i in range(epochs):
        Optimizer.zero_grad()
        Output = model(Adv)
        Target = torch.full_like(y, Objective)
        
        TargetLoss = MseLoss(Output, Target)
        DiffLoss = MyLoss(Adv, X)
        
        loss = c * TargetLoss +  DiffLoss

        loss.backward()
        Optimizer.step()
        with torch.no_grad():
            Adv.clamp_(0.0, 1.0)
            
    return Adv

def Project(Adv, X, S, Loss):
    L2 = ((Adv - X) ** 2)
    Result = Loss.tanh(L2, Loss.k)
    Result2 = torch.sum(Result, dim=1)
    Result3 = Loss.tanh(Result2, Loss.k)
    
    ToProject = []
    for i in range(len(Result3[0])):
        if Result3[0][i] < S:
            ToProject.append(i)
    
    for Ins in range(len(Adv[0])):
        for i in ToProject:
            Adv[0][Ins][i] = X[0][Ins][i]
    return Adv

def IterativeMaillardL0(model, Objective, X, y, LearningRate, Init, Min, Steps, epochs, k, P, S, device):
    c = Init 
    LInfos = []
    while c > Min:
        Adversarials = MaillardL0(model, Objective, X, y, LearningRate, c, epochs, k, P, S, device)
        Infos = UtilsTool.GetInfos(X, y, 0, Adversarials, model)
        Infos["c"] = c
        LInfos.append(Infos)
        print(c, Infos)
        c/=Steps
    return LInfos


def IterativeMaillardL2(model, Objective, X, y, LearningRate, Init, Min, Steps, epochs, device):
    c = Init
    LInfos = []
    while c > Min:
        Adversarials = MaillardL2(model, Objective, X, y, LearningRate, c, epochs, device)
        Infos = UtilsTool.GetInfos(X, y, 0, Adversarials, model)
        Infos["c"] = c
        LInfos.append(Infos)
        c/=Steps
    return LInfos

def MaillardL0(model, Objective, X, y, LearningRate, c, epochs, k, P, S, device):
    model = model.to(device)
    model.train()
    
    LAdversarial =[]
    for Instance in range(len(X)):
        Adv = X[Instance:Instance+1].clone()
        Adv.requires_grad = True
        Optimizer = optim.Adam([Adv], lr=LearningRate)
        MyLoss = L0Loss(k, device)
        MseLoss = nn.MSELoss()
        
        MinLoss = 100000000
        Target = torch.full_like(y[Instance:Instance+1], Objective)
        SavedAdv = Adv.clone()
        
        for i in range(epochs):
            token = 0
            
            with torch.no_grad():
                Adv.clamp_(0.0, 1.0)
                if random.randint(0, P) == 0:
                    Adv = Project(Adv, X[Instance:Instance+1], S, MyLoss)    
                    token = 1   
            
            Output = model(Adv)
            
            TargetLoss = MseLoss(Output, Target)
            DiffLoss = MyLoss(Adv, X[Instance:Instance+1])
            
            loss = c * TargetLoss +  DiffLoss
            
            if token == 1 and loss.item() < MinLoss:
                MinLoss = loss.item()
                SavedAdv = Adv.clone()
            
            Optimizer.zero_grad()
            loss.backward()
            Optimizer.step()
            
        LAdversarial.append(SavedAdv) 
    FAdversarial = torch.cat(LAdversarial, dim=0)
    return FAdversarial

In [18]:
def GetL0(X, Z):
    L2 = ((X - Z) ** 2)
    Result = torch.sum(L2, dim=1)
    Result = torch.sum(Result != 0, dim=1)
    Result = Result.float().mean()
    return Result

def DisplayCgraph(A, B, C, MainLabel, label1, label2):
    fig, ax1 = plt.subplots()
    ax1.set_xscale('log')

    ax1.plot(A, B, 'b-', label=label1)
    ax1.set_xlabel('c')
    
    ax1.set_ylabel(label1, color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(A, C, 'r-', label=label2)
    
    ax2.set_ylabel(label2, color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title(MainLabel)
    fig.tight_layout()

    plt.show()

def TestNewAttacks(model, X, y, AttacksParameters, device):
    X, y = X.to(device).to(torch.float32), y.to(device).to(torch.float32)
    LInfos = IterativeMaillardL2(model, AttacksParameters["Maillard"]["Objective"], X, y, AttacksParameters["Maillard"]["LearningRate"], AttacksParameters["Maillard"]["Initial_c"], AttacksParameters["Maillard"]["Final_c"], AttacksParameters["Maillard"]["Steps_c"], AttacksParameters["Maillard"]["Iterations"], device)
    A = [item['c'] for item in LInfos]
    B = [item['PredRUL'] for item in LInfos]
    C = [item['RMSE_adversarial'] for item in LInfos]
    DisplayCgraph(A, B, C, 'PredRUL and RMSE evolution', 'RUL', 'RMSE')
    
    LInfos2 = IterativeMaillardL0(model, AttacksParameters["Maillard"]["Objective"], X, y, AttacksParameters["Maillard"]["LearningRate"], AttacksParameters["Maillard"]["Initial_c"], AttacksParameters["Maillard"]["Final_c"], AttacksParameters["Maillard"]["Steps_c"], AttacksParameters["Maillard"]["Iterations"], AttacksParameters["Maillard"]["k"], AttacksParameters["Maillard"]["P"], AttacksParameters["Maillard"]["S"], device)
    A = [item['c'] for item in LInfos2]
    B = [item['PredRUL'] for item in LInfos2]
    C = [item['L0'] for item in LInfos2]
    DisplayCgraph(A, B, C, ' RUL and L0 evolution', 'RUL', 'L0')

In [19]:
if OptimizeParameters == 1:
    KList = {0.3, 0.6, 1, 5, 10}
    FixP = 5
    FixS = 0.01
    Epochs = 1000

    c = 1
    X, y = X.to(device).to(torch.float32), y.to(device).to(torch.float32)

    with open(OptimalParametersPath + "OptimizeK.txt", "w") as file:
        while c > 0.0001:
            for k in KList:
                CurrentAdversarial = MaillardL0(MyGRUModel, 0, X, y, 0.01, c, Epochs, k, FixP, FixS, device)
                info = UtilsTool.GetInfos(X, y, 0, CurrentAdversarial, MyGRUModel)
                result = f"{c}, {k}, {info}\n"
                file.write(result)
                file.flush()
                print(c, k, result)
            print()
            c/=2
        
if OptimizeParameters == 1:
    FixK = 1
    PList = [1,2,5,8]
    SList = [0.003, 0.01, 0.03, 0.1]
    Epochs = 1000

    c = 1
    X, y = X.to(device).to(torch.float32), y.to(device).to(torch.float32)
    with open(OptimalParametersPath + "OptimizePS.txt", "w") as file:
        while c > 0.0001:
            for P in PList:
                for S in SList:
                    CurrentAdversarial = MaillardL0(MyGRUModel, 0, X, y, 0.01, c, Epochs, FixK, P, S, device)
                    info = UtilsTool.GetInfos(X, y, 0, CurrentAdversarial, MyGRUModel)
                    result = f"{c}, {P}, {S}, {info}\n"
                    file.write(result)
                    file.flush()
                    print(result)
            print()
            c/=2

1 0.3 1, 0.3, {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 2.359086751937866, 'RMSE_adversarial': 0.10449633747339249, 'RMSE_pred': 83.73699188232422, 'L0': tensor(9.0600)}

1 0.6 1, 0.6, {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.323388010263443, 'RMSE_adversarial': 0.06320112943649292, 'RMSE_pred': 85.9336166381836, 'L0': tensor(11.3400)}

1 1 1, 1, {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.21787388622760773, 'RMSE_adversarial': 0.07218281179666519, 'RMSE_pred': 86.02108764648438, 'L0': tensor(10.6700)}

1 5 1, 5, {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.15059970319271088, 'RMSE_adversarial': 0.09399057179689407, 'RMSE_pred': 86.07726287841797, 'L0': tensor(22.6800)}

1 10 1, 10, {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.14717990159988403, 'RMSE_adversarial': 0.09404253214597702, 'RMSE_pred': 86.07791137695312, 'L0': tensor(23.3500)}




In [ ]:
"""
if OptimizeParameters == 1:
    FixK = 1
    PList = [1,2,5,8]
    SList = [0.003, 0.01, 0.03, 0.1]

    c = 1
    TX, Ty = X[0:5], y[0:5]
    TX, Ty = TX.to(device).to(torch.float32), Ty.to(device).to(torch.float32)
    while c > 0.0001:
        for P in PList:
            for S in SList:
                CurrentAdversarial = MaillardL0(MyGRUModel, 0, TX, Ty, 0.01, c, 1000, FixK, P, S, device)
                print(c, P, S , UtilsTool.GetInfos(TX, Ty, 0, CurrentAdversarial, MyGRUModel))
        print()
        c/=1.5
"""

'\nif OptimizeParameters == 1:\n    FixK = 1\n    PList = [1,2,5,8]\n    SList = [0.003, 0.01, 0.03, 0.1]\n\n    c = 1\n    TX, Ty = X[0:5], y[0:5]\n    TX, Ty = TX.to(device).to(torch.float32), Ty.to(device).to(torch.float32)\n    while c > 0.0001:\n        for P in PList:\n            for S in SList:\n                CurrentAdversarial = MaillardL0(MyGRUModel, 0, TX, Ty, 0.01, c, 1000, FixK, P, S, device)\n                print(c, P, S , UtilsTool.GetInfos(TX, Ty, 0, CurrentAdversarial, MyGRUModel))\n        print()\n        c/=1.5\n'

In [ ]:
if DisplayGraph == 1:
    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.0001,
            "Steps_c":2,
            "k": 5,
            "P": 5,
            "S": 0.03

        }
    }
    TestNewAttacks(MyRNNModel, X, y, AttacksParametersMaillard, device)

In [ ]:
if DisplayGraph == 1:
    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.0001,
            "Steps_c":2,
            "k": 5,
            "P": 5,
            "S": 0.03

        }
    }
    TX, Ty = X[0:3], y[0:3]
    TestNewAttacks(MyGRUModel, TX, Ty, AttacksParametersMaillard, device)

In [ ]:
if DisplayGraph == 1:
    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.0001,
            "Steps_c":2,
            "k": 5,
            "P": 5,
            "S": 0.03

        }
    }
    TestNewAttacks(MyLSTMModel, X, y, AttacksParametersMaillard, device)

In [ ]:
RNNAdversarialDataMaillardL2 = MaillardL2(MyRNNModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
GRUAdversarialDataMaillardL2 = MaillardL2(MyGRUModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
LSTMAdversarialDataMaillardL2 = MaillardL2(MyLSTMModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)

In [ ]:
RNNAdversarialDataMaillardL0 = MaillardL0(MyRNNModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 1, 5, 0.03, device)
GRUAdversarialDataMaillardL0 = MaillardL0(MyGRUModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 1, 5, 0.03, device)
LSTMAdversarialDataMaillardL0 = MaillardL0(MyLSTMModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 1, 5, 0.03, device)

In [ ]:
def DisplayResults(testloaderAttack, X, AdvX, y, minmax_dict, device):
    X, y = X.to(device).to(torch.float32), y.to(device).to(torch.float32)
    
    columns_to_update = ['os1', 'os2', 'os3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
    CurrentDataset = testloaderAttack.dataset

    for i in range(len(X)):
        DF = CurrentDataset.__getdf__(i)
        NDF = DataTool.UnnormalizeDataset(DF, minmax_dict)
        
        DF[columns_to_update] = AdvX[i].cpu().detach().numpy()
        ADF = DataTool.UnnormalizeDataset(DF, minmax_dict)
        
        print(NDF)
        print(ADF)

In [ ]:
def round_column(series, decimal_places):
    return series.apply(lambda x: float(Decimal(str(x)).quantize(Decimal('1e-{0}'.format(decimal_places)), rounding=ROUND_HALF_UP)))

def ApplyDiscrete(DF, minmax_dict):
    for c in DF.columns:
        if c + 'unit' in minmax_dict:
            Dec = minmax_dict[c + 'unit']
            if math.isnan(Dec):
                Dec = 0
            DF[c] = round_column(DF[c], Dec)       
    return DF 

In [ ]:
def TestProjectionCost(testloaderAttack, X, Adversarial, MyModel):
    columns_to_update = ['os1', 'os2', 'os3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
    ListADF = []

    CurrentDataset = testloaderAttack.dataset
    for i in range(len(Adversarial)):
        DF = CurrentDataset.__getdf__(i)
        DF[columns_to_update] = Adversarial[i].cpu().detach().numpy()
        ADF = DataTool.UnnormalizeDataset(DF, minmax_dict)
        PDF = ApplyDiscrete(ADF, minmax_dict)
        NPDF = DataTool.NormalizeDataset(PDF, minmax_dict)
        ListADF.append(NPDF)

    FADF = pd.concat(ListADF, ignore_index=True)
    FADF = FADF.drop('index', axis=1)
    
    testdataset2 = DataTool.test(units, FADF)
    testloader2 = DataLoader(testdataset2, batch_size = 100)
    
    for Adv, y in testloader2:
        X, Adv, y = X.to(device).to(torch.float32), Adv.to(device).to(torch.float32), y.to(device).to(torch.float32)
        Infos = UtilsTool.GetInfos(X, y, 0, Adversarial, MyModel)
        print("Initial:", Infos)
        Infos = UtilsTool.GetInfos(X, y, 0, Adv, MyModel)
        print("Formated:", Infos)
    print()
    

In [ ]:
print("RNN")
TestProjectionCost(testloaderAttack, X, RNNAdversarialDataFgsm, MyRNNModel)
TestProjectionCost(testloaderAttack, X, RNNAdversarialDataBim, MyRNNModel)
TestProjectionCost(testloaderAttack, X, RNNAdversarialDataCW, MyRNNModel)
TestProjectionCost(testloaderAttack, X, RNNAdversarialDataMaillardL2, MyRNNModel)
TestProjectionCost(testloaderAttack, X, RNNAdversarialDataMaillardL0, MyRNNModel)

RNN
Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 47.80316162109375, 'RMSE_adversarial': 0.09092852473258972, 'RMSE_pred': 42.452213287353516, 'L0': tensor(23.7200)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 49.51344680786133, 'RMSE_adversarial': 0.08046704530715942, 'RMSE_pred': 40.866058349609375, 'L0': tensor(16.)}

Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 28.27294158935547, 'RMSE_adversarial': 0.094657301902771, 'RMSE_pred': 60.857696533203125, 'L0': tensor(23.7000)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 31.377830505371094, 'RMSE_adversarial': 0.08366050571203232, 'RMSE_pred': 58.04108810424805, 'L0': tensor(16.)}

Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 10.619772911071777, 'RMSE_adversarial': 0.10618610680103302, 'RMSE_pred': 77.25652313232422, 'L0': tensor(23.9800)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 14.5673828125, 'RMSE_adve

In [ ]:
print("GRU")
TestProjectionCost(testloaderAttack, X, GRUAdversarialDataFgsm, MyGRUModel)
TestProjectionCost(testloaderAttack, X, GRUAdversarialDataBim, MyGRUModel)
TestProjectionCost(testloaderAttack, X, GRUAdversarialDataCW, MyGRUModel)
TestProjectionCost(testloaderAttack, X, GRUAdversarialDataMaillardL2, MyRNNModel)
TestProjectionCost(testloaderAttack, X, GRUAdversarialDataMaillardL0, MyRNNModel)

GRU
Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 22.89347267150879, 'RMSE_adversarial': 0.0545102022588253, 'RMSE_pred': 61.21987533569336, 'L0': tensor(23.9400)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 23.43755531311035, 'RMSE_adversarial': 0.051656171679496765, 'RMSE_pred': 60.758182525634766, 'L0': tensor(16.)}

Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 8.817352294921875, 'RMSE_adversarial': 0.0562066026031971, 'RMSE_pred': 76.15291595458984, 'L0': tensor(23.8600)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 10.106322288513184, 'RMSE_adversarial': 0.051005925983190536, 'RMSE_pred': 74.8314208984375, 'L0': tensor(16.)}

Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.2721460163593292, 'RMSE_adversarial': 0.055462464690208435, 'RMSE_pred': 85.945068359375, 'L0': tensor(23.8300)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 0.630379855632782, 'RMSE_ad

In [ ]:
print("LSTM")
TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataFgsm, MyLSTMModel)
TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataBim, MyLSTMModel)
TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataCW, MyLSTMModel)
TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataMaillardL2, MyRNNModel)
TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataMaillardL0, MyRNNModel)

LSTM
Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 38.80435562133789, 'RMSE_adversarial': 0.13434146344661713, 'RMSE_pred': 51.77580261230469, 'L0': tensor(23.4300)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 38.15455627441406, 'RMSE_adversarial': 0.1235138475894928, 'RMSE_pred': 51.54292678833008, 'L0': tensor(16.)}

Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 17.44858169555664, 'RMSE_adversarial': 0.12679541110992432, 'RMSE_pred': 69.95494079589844, 'L0': tensor(22.3000)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 20.22705078125, 'RMSE_adversarial': 0.1142820492386818, 'RMSE_pred': 67.28006744384766, 'L0': tensor(16.)}

Initial: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 9.360200881958008, 'RMSE_adversarial': 0.11699812859296799, 'RMSE_pred': 77.15238952636719, 'L0': tensor(22.8700)}
Formated: {'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 11.559171676635742, 'RMSE_advers

In [ ]:
print("Transferability RNN")
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataFgsm, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataBim, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataCW, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL2, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL0, MyRNNModel)
print(Infos)
print()
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataFgsm, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataBim, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataCW, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL2, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL0, MyRNNModel)
print(Infos)
print()
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataFgsm, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataBim, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataCW, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL2, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL0, MyRNNModel)
print(Infos)

Transferability RNN
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 47.80316162109375, 'RMSE_adversarial': 0.09092852473258972, 'RMSE_pred': 42.452213287353516, 'L0': tensor(23.7200)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 28.27294158935547, 'RMSE_adversarial': 0.094657301902771, 'RMSE_pred': 60.857696533203125, 'L0': tensor(23.7000)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 10.619772911071777, 'RMSE_adversarial': 0.10618610680103302, 'RMSE_pred': 77.25652313232422, 'L0': tensor(23.9800)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 14.913652420043945, 'RMSE_adversarial': 0.07828199863433838, 'RMSE_pred': 73.8879165649414, 'L0': tensor(23.9400)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 27.584379196166992, 'RMSE_adversarial': 0.08975664526224136, 'RMSE_pred': 66.11058807373047, 'L0': tensor(4.3200)}

{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 58.03468704223633, 'RMSE_adversarial': 0.0545102022588253, 'R

In [ ]:
print("Transferability GRU")
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataFgsm, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataBim, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataCW, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL2, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL0, MyGRUModel)
print(Infos)
print()
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataFgsm, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataBim, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataCW, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL2, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL0, MyGRUModel)
print(Infos)
print()
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataFgsm, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataBim, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataCW, MyGRUModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL2, MyRNNModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL0, MyRNNModel)
print(Infos)

Transferability GRU
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 22.153730392456055, 'RMSE_adversarial': 0.09092852473258972, 'RMSE_pred': 66.33668518066406, 'L0': tensor(23.7200)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 27.141986846923828, 'RMSE_adversarial': 0.094657301902771, 'RMSE_pred': 63.67589569091797, 'L0': tensor(23.7000)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 25.213661193847656, 'RMSE_adversarial': 0.10618610680103302, 'RMSE_pred': 65.58560943603516, 'L0': tensor(23.9800)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 29.597238540649414, 'RMSE_adversarial': 0.07828199863433838, 'RMSE_pred': 58.66758346557617, 'L0': tensor(23.9400)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 34.82459259033203, 'RMSE_adversarial': 0.08975664526224136, 'RMSE_pred': 65.16228485107422, 'L0': tensor(4.3200)}

{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 22.89347267150879, 'RMSE_adversarial': 0.0545102022588253, 'R

In [ ]:
print("Transferability LSTM")

Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataFgsm, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataBim, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataCW, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL2, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL0, MyLSTMModel)
print(Infos)
print()
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataFgsm, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataBim, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataCW, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL2, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL0, MyLSTMModel)
print(Infos)
print()
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataFgsm, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataBim, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataCW, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL2, MyLSTMModel)
print(Infos)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL0, MyLSTMModel)
print(Infos)

Transferability LSTM
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 42.12148666381836, 'RMSE_adversarial': 0.09092852473258972, 'RMSE_pred': 48.099124908447266, 'L0': tensor(23.7200)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 32.42097854614258, 'RMSE_adversarial': 0.094657301902771, 'RMSE_pred': 58.23992156982422, 'L0': tensor(23.7000)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 41.434242248535156, 'RMSE_adversarial': 0.10618610680103302, 'RMSE_pred': 51.004642486572266, 'L0': tensor(23.9800)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 44.02229309082031, 'RMSE_adversarial': 0.07828199863433838, 'RMSE_pred': 48.177852630615234, 'L0': tensor(23.9400)}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 40.557334899902344, 'RMSE_adversarial': 0.08975664526224136, 'RMSE_pred': 54.969444274902344, 'L0': tensor(4.3200)}

{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 40.85871124267578, 'RMSE_adversarial': 0.0545102022588253,

In [ ]:
def exp_degradation(x, D0, lambda_):
    return D0 * np.exp(lambda_ * x)

def prepare_batch(X_batch, y_batch):
    X_batch_np = X_batch.view(X_batch.size(0), -1).numpy()
    y_batch_np = y_batch.numpy()
    
    X_batch_np = np.mean(X_batch_np, axis=1)
    
    return X_batch_np, y_batch_np

def prepare_data(loader):
    X_list = []
    y_list = []
    for X_batch, y_batch in loader:
        X_list.append(X_batch.view(X_batch.size(0), -1).numpy())
        y_list.append(y_batch.numpy())
    X_np = np.vstack(X_list)
    y_np = np.hstack(y_list)
    
    X_np = np.mean(X_np, axis=1)
    
    return X_np, y_np

In [ ]:
def GetStatsInfos(X, y, Objective, AX, Parameters):
    X, y, AX = X.cpu(), y.cpu(), AX.cpu()
    rmse_adversarials = UtilsTool.PredRmse(X, AX).item()
    X_adv_np, y_adv_np = prepare_batch(AX.detach(), y)
    output = exp_degradation(X_adv_np, *Parameters)
    average = torch.mean(torch.tensor(output).float()).item()
    averagey = torch.mean(y.float()).item()
    rmse_pred = UtilsTool.PredRmse(y.float(), torch.tensor(output).float()).item()
    extra_info = {
        'RealRUL': averagey,
        'Objective': Objective,
        'PredRUL': average,
        'RMSE_adversarial': rmse_adversarials,
        'RMSE_pred': rmse_pred
    }
    return extra_info

In [ ]:
def TrainStatModel(trainloader):
    X_train_np, y_train_np = prepare_data(trainloader)
    Param, pcov = curve_fit(exp_degradation, X_train_np, y_train_np, maxfev=10000)
    return Param

Param = TrainStatModel(trainloader)

In [ ]:
print(GetStatsInfos(X, y, 0, X, Param))

{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 98.66363525390625, 'RMSE_adversarial': 0.0, 'RMSE_pred': 34.968929290771484}


In [ ]:
if DisplayGraph == 1:
    UtilsTool.DisplayGraph(torch.tensor(y_pred.cpu()), y.cpu())

In [ ]:
print(GetStatsInfos(X, y, 0, RNNAdversarialDataFgsm,  Param))
print(GetStatsInfos(X, y, 0, RNNAdversarialDataBim, Param))
print(GetStatsInfos(X, y, 0, RNNAdversarialDataCW, Param))
print(GetStatsInfos(X, y, 0, RNNAdversarialDataMaillardL0, Param))
print(GetStatsInfos(X, y, 0, RNNAdversarialDataMaillardL2, Param))
print()
print(GetStatsInfos(X, y, 0, GRUAdversarialDataFgsm,  Param))
print(GetStatsInfos(X, y, 0, GRUAdversarialDataBim, Param))
print(GetStatsInfos(X, y, 0, GRUAdversarialDataCW, Param))
print(GetStatsInfos(X, y, 0, GRUAdversarialDataMaillardL0, Param))
print(GetStatsInfos(X, y, 0, GRUAdversarialDataMaillardL2, Param))
print()
print(GetStatsInfos(X, y, 0, LSTMAdversarialDataFgsm,  Param))
print(GetStatsInfos(X, y, 0, LSTMAdversarialDataBim, Param))
print(GetStatsInfos(X, y, 0, LSTMAdversarialDataCW, Param))
print(GetStatsInfos(X, y, 0, LSTMAdversarialDataMaillardL0, Param))
print(GetStatsInfos(X, y, 0, LSTMAdversarialDataMaillardL2, Param))

{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 72.06945037841797, 'RMSE_adversarial': 0.09092851728200912, 'RMSE_pred': 30.323530197143555}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 63.09111785888672, 'RMSE_adversarial': 0.09465730935335159, 'RMSE_pred': 40.2687873840332}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 95.46247100830078, 'RMSE_adversarial': 0.10618611425161362, 'RMSE_pred': 34.7204475402832}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 95.914794921875, 'RMSE_adversarial': 0.08975664526224136, 'RMSE_pred': 34.67224884033203}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 95.5511703491211, 'RMSE_adversarial': 0.07828200608491898, 'RMSE_pred': 33.29011535644531}

{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 79.91079711914062, 'RMSE_adversarial': 0.0545102059841156, 'RMSE_pred': 28.2359619140625}
{'RealRUL': 75.5199966430664, 'Objective': 0, 'PredRUL': 71.84984588623047, 'RMSE_adversarial': 0.0562066100537